<a href="https://colab.research.google.com/github/tjturnage/satellite/blob/master/Satellite_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>This code borrows heavily from</h2>

<ul>
<li>this <a href="https://lsterzinger.medium.com/add-lat-lon-coordinates-to-goes-16-goes-17-l2-data-and-plot-with-cartopy-27f07879157f" target="_blank">medium article</a> showing how to slice satellite data by lat/lon coordinates</li>
<li>The <a href="https://goes2go.readthedocs.io/en/latest/" target="_blank">goes2go</a> library developed by <a href="https://github.com/blaylockbk" target="_blank">Brian Blaylock</a> including <a href="https://goes2go.readthedocs.io/en/latest/reference_guide/index.html#rgb-recipes" target="_blank">RGB recipes</a></li>
</ul>
<br>
<h4>As of now, it's hardwired to plot 6-panel figures that's defined by the
editable products list below.</h4>
<hr>

In [1]:
start_date = None
start_hour = None
start_minute = None
end_date = None
end_hour = None
end_minute = None
# @markdown # Choose plot options
plot_counties = True # @param {type:"boolean"}
plot_roads = False  # @param {type:"boolean"}

# @markdown # Define plot boundaries in lat/lon coordinates
west_edge = -91.0 # @param {type:"number"}
east_edge = -80.0 # @param {type:"number"}
south_edge = 41.0 # @param {type:"number"}
north_edge = 46.5 # @param {type:"number"}

plot_extent = [west_edge, east_edge, south_edge, north_edge]
# @markdown # Define start and end times to plot

# @markdown ## ---------  start date/time  -----------
start_date = "2023-08-24" # @param {type:"date"}
start_hour = 22 # @param {type:"slider", min:0, max:23, step:1}
start_minute = "45" # @param ["00","15","30","45"] {type:"string"}
# @markdown ## ---------  end date/time  -------------
end_date = "2023-08-25" # @param {type:"date"}
end_hour = 3 # @param {type:"slider", min:0, max:23, step:1}
end_minute = "15" # @param ["00","15","30","45"] {type:"string"}

if len(str(start_hour)) == 1:
  start_hour = f"0{start_hour}"
if len(str(end_hour)) == 1:
  end_hour = f"0{end_hour}"

data_start = f"{start_date} {start_hour}:{start_minute}"
data_end = f"{end_date} {end_hour}:{end_minute}"
#data_start='2023-08-24 22:00'
#data_end='2023-08-24 23:00'


# defines lat/lon range to slice data for plotting
# a larger area is needed for extra margin since the
# plot area gets sheared and warped # during reprojection
lons = (plot_extent[0] - 1.5, plot_extent[1] + 1.5)
lats = (plot_extent[2] - 1.0, plot_extent[3] + 1.0)

if plot_extent[1] - plot_extent[0] > 8:
  plot_counties = False

<h2><b>ABI Band References</b></h2>

*   http://cimss.ssec.wisc.edu/goes/GOESR_QuickGuides.html
*   https://www.weather.gov/media/crp/GOES_16_Guides_FINALBIS.pdf
*   https://rammb2.cira.colostate.edu/training/visit/quick_reference/#tab17
*   https://www.goes-r.gov/mission/ABI-bands-quick-info.html


<h2><b>Select 6 Bands and/or RGBs to plot</b></h2>
<h4>Individual Channels</h4>
<table>

<tr>
<td>C01</td>
<td>"Blue" Visible</td>
</tr>
<tr>
<td>C02</td>
<td>"Red" Visible</td>
</tr>
<tr>
<td>C02i</td>
<td>Hi-Res "Red" Visible ... (best for zoomed in displays)</td>
</tr>
<tr>
<td>C03</td>
<td>"Veggie" Near-IR</td>
</tr>
<tr>
<td>C05</td>
<td>"Snow/Ice" Near-IR</td>
</tr>
<tr>
<td>C08</td>
<td>"Upper-Level Tropospheric Water Vapor" IR</td>
</tr>
<tr>
<td>C09</td>
<td>"Mid-Level Tropospheric Water Vapor" IR</td>
</tr>
<tr>
<td>C10</td>
<td>"Lower-Level Tropospheric Water Vapor" IR</td>
</tr>
<tr>
<td>C13</td>
<td>"Clean" IR Longwave Window</td>
</tr>
<tr>
<td>C14</td>
<td>IR Longwave Window</td>
</tr>
<tr>
<td>C15</td>
<td>"Dirty" IR Longwave</td>
</tr>
</table>

<h4>RGBs</h4>
<table>
<tr>
<td>Airmass</td>
</tr>
<tr>
<td>NightMicrophysics</td>
</tr>
<tr>
<td>DifferentialWV</td>
</tr>
<tr>
<td>SplitWindowDifference</td>
</tr>
</table>


In [2]:
products = ['C08', 'C09', 'C10', 'AirMass', 'C13', 'C02i']

In [3]:
# @title Install needed packages
print("Installing Packages ...")

import warnings
warnings.filterwarnings('ignore')
!pip install s3fs &> /dev/null
!pip install pyproj &> /dev/null
!pip install xarray &> /dev/null
!pip install cartopy &> /dev/null
!pip install MetPy &> /dev/null
!pip install goes2go &> /dev/null

print("Installations Complete!")

import os
import s3fs
from pyproj import Proj
import xarray as xr
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime,timezone
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
import matplotlib as mpl

print("Library Imports Complete!")

Installing Packages ...
Installations Complete!
Library Imports Complete!


In [4]:
# @title Create colormaps then acquire county and road shapefiles
def create_and_register_cmap(cmap_name, original_colors, position, rgb_input=True):
    """
    1) Converts list of color values from 8-bit (0-255 scale)
       to 0-1 decimal scale.

    2) The created final_color_list is combine with position list
       to create a matplotlib colormap.

    3) Registers the new colormap using the supplied cmap_name.

    Input
    ----------
             cmap_name : string
                         Name of the cmap you want to use and register

      original_ colors : list
                         8-bit tuples containing RGB values
                         example: [(127,256,192), (125,125,125)]

              position : list
                         node locations for color break points on
                         a 0 to 1 scale

            rgb_input : boolean
                        True: Color conversion from RGB to performed.
                       False: Color conversion from RGB is skipped.

    Returns
    -------
    None

    """
    if rgb_input:
        final_color_list = []
        for color in original_colors:
            converted_color_group = []
            for original_channel_value in color:
                new_channel_value = float(original_channel_value)/255
                converted_color_group.append(float(format(new_channel_value, '.3f')))
            converted_color_tuple = tuple(converted_color_group)
            final_color_list.append(converted_color_tuple)
    else:
        final_color_list = original_colors

    this_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(position, final_color_list)))
    mpl.colormaps.register(cmap=this_cmap, force=True)
    return


###############################################################################

#---------------  CH13 (clean IR) ----------------
# Using COD version
ir_colors = [(0,0,0),(255,255,255),(255,0,0), (255,255,0), (0,255,0), (0,0,255),
 (191,0,255), (255,255,255),(120,120,120),(0,0,0)]
ir_nodes = [0, 10/166, 45/166, 55/166, 65/166, 82/166, 90/166, 95/166, 136/166,
            1]
create_and_register_cmap('cod_ir', ir_colors, ir_nodes)

#--------------- WV CH08,CH09,CH10 ---------------
wv_colors = [(0,255,255),(0,110,0),(255,255,255),(0,0,165),(255,255,0),
 (255,0,0),(0, 0, 0)]
wv_nodes = [0, (109-75)/109,(109-47)/109, (109-30)/109, (109-15.5)/109,
            108/109,1 ]
create_and_register_cmap('wv_cmap', wv_colors, wv_nodes)


#--------------- Winter Ref ----------------------
winter_rgbs = [(0, 0, 0), (0, 0, 0), (70, 70, 70), (255, 255, 255),
(0, 80, 255), (0, 0, 160), (0, 0, 60), (255, 255, 0), (220, 100, 0),
(220, 100, 0), (220, 100, 0)]
winter_nodes = [0.0, 0.0984, 0.2213, 0.3361, 0.3689, 0.4098, 0.4918, 0.5246,
                0.5902, 0.918, 1.0]
create_and_register_cmap('winter_z', winter_rgbs, winter_nodes)

#--------------- WDTD Ref ----------------------
wdtd_z_rgbs =[(0, 0, 0), (0, 0, 0), (130, 130, 130), (95, 189, 207),
(57, 201, 105), (57, 201, 105), (0, 40, 0), (9, 94, 9), (255, 207, 0),
(255, 207, 0), (255, 133, 0), (255, 0, 0), (89, 0, 0), (255, 245, 255),
(225, 11, 227), (164, 0, 247), (99, 0, 214), (5, 221, 224), (58, 103, 181),
(255, 255, 255), (255, 255, 255)]
wdtd_z_nodes = [0.0, 0.0164, 0.3844, 0.3852, 0.4254, 0.4262, 0.5484, 0.5492,
                0.5893, 0.5902, 0.6713, 0.6721, 0.7533, 0.7541, 0.8352, 0.8361,
                0.8762, 0.877, 0.9172, 0.918, 1.0]
create_and_register_cmap('wdtd_z', wdtd_z_rgbs, wdtd_z_nodes)


################################################################################

import urllib.request
import requests
from bs4 import BeautifulSoup
import zipfile

url = "https://www.weather.gov/gis/Counties"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
zip_links = soup.find_all("a", href=lambda href: href.endswith(".zip"))
if zip_links:
    zip_url = zip_links[-1]["href"]
    counties_url = os.path.join('https://www.weather.gov',str(zip_url)[1:])
    zip_file_name = counties_url.split("/")[-1]
    zip_file_root = zip_file_name[:-4]
    shapefile_name = f'{zip_file_root}.shp'
else:
    print("No zip files found on the website.")

if os.path.exists('/content/counties.zip') is False:
  urllib.request.urlretrieve(counties_url, "counties.zip")
with zipfile.ZipFile('counties.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

try:
  COUNTY_SHAPEFILE
except NameError:
  reader = shpreader.Reader(shapefile_name)
  features = list(reader.geometries())
  COUNTY_SHAPEFILE = cfeature.ShapelyFeature(features, ccrs.PlateCarree())

# download and read road shapefile
if os.path.exists('/content/roads.zip') is False:
  road_url = "https://www2.census.gov/geo/tiger/TIGER2019/PRIMARYROADS/tl_2019_us_primaryroads.zip"
  urllib.request.urlretrieve(road_url, "roads.zip")
with zipfile.ZipFile('roads.zip', 'r') as zip_roads:
    zip_roads.extractall('.')
try:
  ROAD_SHAPEFILE
except NameError:
  road_reader = shpreader.Reader("tl_2019_us_primaryroads.shp")
  features = list(road_reader.geometries())
  ROAD_SHAPEFILE = cfeature.ShapelyFeature(features, ccrs.PlateCarree())

In [5]:
# @title Define functions for lat/lon coordinates and satellite RGBs

#---------------------------------------------------------------------
#   The following functions that calculate lat/lon coordinate values and then add
#   these dimensions to the xarray come from:
#
#   https://lsterzinger.medium.com/add-lat-lon-coordinates-to-goes-16-goes-17-l2-data-and-plot-with-cartopy-27f07879157f
#---------------------------------------------------------------------

def calc_latlon(ds):
    # The math for this function was taken from
    # https://makersportal.com/blog/2018/11/25/goes-r-satellite-latitude-and-longitude-grid-projection-algorithm
    x = ds.x
    y = ds.y
    goes_imager_projection = ds.goes_imager_projection

    x,y = np.meshgrid(x,y)

    r_eq = goes_imager_projection.attrs["semi_major_axis"]
    r_pol = goes_imager_projection.attrs["semi_minor_axis"]
    l_0 = goes_imager_projection.attrs["longitude_of_projection_origin"] * (np.pi/180)
    h_sat = goes_imager_projection.attrs["perspective_point_height"]
    H = r_eq + h_sat

    a = np.sin(x)**2 + (np.cos(x)**2 * (np.cos(y)**2 + (r_eq**2 / r_pol**2) * np.sin(y)**2))
    b = -2 * H * np.cos(x) * np.cos(y)
    c = H**2 - r_eq**2

    r_s = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)

    s_x = r_s * np.cos(x) * np.cos(y)
    s_y = -r_s * np.sin(x)
    s_z = r_s * np.cos(x) * np.sin(y)

    lat = np.arctan((r_eq**2 / r_pol**2) * (s_z / np.sqrt((H-s_x)**2 +s_y**2))) * (180/np.pi)
    lon = (l_0 - np.arctan(s_y / (H-s_x))) * (180/np.pi)



    ds = ds.assign_coords({
        "lat":(["y","x"],lat),
        "lon":(["y","x"],lon)
    })
    ds.lat.attrs["units"] = "degrees_north"
    ds.lon.attrs["units"] = "degrees_east"
    return ds


def get_xy_from_latlon(ds, lats, lons):

    lat1, lat2 = lats
    lon1, lon2 = lons

    lat = ds.lat.data
    lon = ds.lon.data

    x = ds.x.data
    y = ds.y.data

    x,y = np.meshgrid(x,y)

    x = x[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]
    y = y[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]

    return ((min(x), max(x)), (min(y), max(y)))


#---------------------------------------------------------------------
#   The following functions were adapted from:
#   https://github.com/blaylockbk/goes2go/blob/main/goes2go/rgb.py
#---------------------------------------------------------------------

def normalize(value, lower_limit, upper_limit, clip=True):
    """
    Normalize values between 0 and 1.

    Normalize between a lower and upper limit. In other words, it
    converts your number to a value in the range between 0 and 1.
    Follows `normalization formula
    <https://stats.stackexchange.com/a/70807/220885>`_

    This is the same concept as `contrast or histogram stretching
    <https://staff.fnwi.uva.nl/r.vandenboomgaard/IPCV20162017/LectureNotes/IP/PointOperators/ImageStretching.html>`_


    .. code:: python

        NormalizedValue = (OriginalValue-LowerLimit)/(UpperLimit-LowerLimit)

    Parameters
    ----------
    value :
        The original value. A single value, vector, or array.
    upper_limit :
        The upper limit.
    lower_limit :
        The lower limit.
    clip : bool
        - True: Clips values between 0 and 1 for RGB.
        - False: Retain the numbers that extends outside 0-1 range.
    Output:
        Values normalized between the upper and lower limit.
    """
    norm = (value - lower_limit) / (upper_limit - lower_limit)
    if clip:
        norm = np.clip(norm, 0, 1)
    return norm


def AirMass(c08,c10,c12,c13):
    """
    Air Mass RGB:
    (See `Quick Guide <http://rammb.cira.colostate.edu/training/visit/quick_guides/QuickGuide_GOESR_AirMassRGB_final.pdf>`__ for reference)

    .. image:: /_static/AirMass.png

    Parameters
    ----------
    C : xarray.Dataset
        A GOES ABI multichannel file opened with xarray.
    \*\*kwargs :
        Keyword arguments for ``rgb_as_dataset`` function.
        - latlon : derive latitude and longitude of each pixel

    """
    # Load the three channels into appropriate R, G, and B variables
    R = c08 - c10
    G = c12 - c13
    B = c08

    # Normalize each channel by the appropriate range of values. e.g. R = (R-minimum)/(maximum-minimum)
    R = normalize(R, -26.2, 0.6)
    G = normalize(G, -42.2, 6.7)
    B = normalize(B, -64.65, -29.25)

    # Invert B
    B = 1 - B

    # The final RGB array :)
    RGB = np.dstack([R, G, B])
    return RGB

def DayCloudPhase(c13, c02, c05):
    """
    Day Cloud Phase Distinction RGB:
    (See `Quick Guide <http://rammb.cira.colostate.edu/training/visit/quick_guides/Day_Cloud_Phase_Distinction.pdf>`__ for reference)

    .. image:: /_static/DayCloudPhase.png

    Parameters
    ----------
    C : xarray.Dataset
        A GOES ABI multichannel file opened with xarray.
    \*\*kwargs :
        Keyword arguments for ``rgb_as_dataset`` function.
        - latlon : derive latitude and longitude of each pixel

    """
    # Load the three channels into appropriate R, G, and B variables
    R = c13 + 273.15
    G = c02
    B = c05

    # Normalize each channel by the appropriate range of values. (Clipping happens inside function)
    R = normalize(R, -53.5, 7.5)
    G = normalize(G, 0, 0.78)
    B = normalize(B, 0.01, 0.59)

    # Invert R
    R = 1 - R

    # The final RGB array :)
    RGB = np.dstack([R, G, B])

    return RGB


def NighttimeMicrophysics(c07,c13,c15):
    """
    Nighttime Microphysics RGB:
    (See `Quick Guide <http://rammb.cira.colostate.edu/training/visit/quick_guides/QuickGuide_GOESR_NtMicroRGB_final.pdf>`__ for reference)

    .. image:: /_static/NighttimeMicrophysics.png

    Parameters
    ----------
    C : xarray.Dataset
        A GOES ABI multichannel file opened with xarray.
    \*\*kwargs :
        Keyword arguments for ``rgb_as_dataset`` function.
        - latlon : derive latitude and longitude of each pixel

    """
    # Load the three channels into appropriate R, G, and B variables
    R = c15 - c13
    G = c13 - c07
    B = c13

    # Normalize values
    R = normalize(R, -6.7, 2.6)
    G = normalize(G, -3.1, 5.2)
    B = normalize(B, -29.6, 19.5)

    # The final RGB array :)
    RGB = np.dstack([R, G, B])

    return RGB

def DifferentialWaterVapor(c08,c10):
    """
    Differential Water Vapor RGB:
    (See `Quick Guide <http://rammb.cira.colostate.edu/training/visit/quick_guides/QuickGuide_GOESR_DifferentialWaterVaporRGB_final.pdf>`__ for reference)

    .. image:: /_static/DifferentialWaterVapor.png

    Parameters
    ----------
    C : xarray.Dataset
        A GOES ABI multichannel file opened with xarray.
    \*\*kwargs :
        Keyword arguments for ``rgb_as_dataset`` function.
        - latlon : derive latitude and longitude of each pixel

    """
    # Load the three channels into appropriate R, G, and B variables.
    R = c10 - c08
    G = c10
    B = c08

    # Normalize each channel by the appropriate range of values. e.g. R = (R-minimum)/(maximum-minimum)
    R = normalize(R, -3, 30)
    G = normalize(G, -60, 5)
    B = normalize(B, -64.65, -29.25)

    # Gamma correction
    Rgamma = 0.2587
    R = np.power(R, 1 / Rgamma)

    GBgamma = 0.4
    G = np.power(G, 1 / GBgamma)
    B = np.power(B, 1 / GBgamma)

    # Invert the colors
    R = 1 - R
    G = 1 - G
    B = 1 - B

    # The final RGB array :)
    RGB = np.dstack([R, G, B])

    return RGB

def SplitWindowDifference(c13, c15):
    """
    Split Window Difference RGB (greyscale):
    (See `Quick Guide <http://cimss.ssec.wisc.edu/goes/OCLOFactSheetPDFs/ABIQuickGuide_SplitWindowDifference.pdf>`__ for reference)

    .. image:: /_static/SplitWindowDifference.png

    Parameters
    ----------
    C : xarray.Dataset
        A GOES ABI multichannel file opened with xarray.
    \*\*kwargs :
        Keyword arguments for ``rgb_as_dataset`` function.
        - latlon : derive latitude and longitude of each pixel

    """
    # Load the three channels into appropriate R, G, and B variables
    data = c15 - c13

    # Normalize values
    data = normalize(data, -10, 10)

    # The final RGB array :)
    RGB = np.dstack([data, data, data])

    return RGB


#---------------------------------------------------------------------
#   These functions are my own :)
#---------------------------------------------------------------------

def extract_channels(subset):
    c01 = subset.CMI_C01.data
    plts['C01']['data'] = c01

    gamma = 2.2
    c02 = subset.CMI_C02.data
    c02gamma = np.power(c02,1/gamma)
    plts['C02']['data'] = c02gamma

    c05 = subset.CMI_C05.data
    plts['C05']['data'] = c05

    c07 = subset.CMI_C07.data - 273.15
    plts['C07']['data'] = c07

    c08 = subset.CMI_C08.data - 273.15
    plts['C08']['data'] = c08

    c09 = subset.CMI_C09.data - 273.15
    plts['C09']['data'] = c09

    c10 = subset.CMI_C10.data - 273.15
    plts['C10']['data'] = c10

    c12 = subset.CMI_C12.data - 273.15
    plts['C12']['data'] = c12

    c13_orig = subset.CMI_C13.data
    c13 = subset.CMI_C13.data - 273.15
    plts['C13']['data'] = c13

    c15_orig = subset.CMI_C15.data
    c15 = subset.CMI_C15.data - 273.15
    plts['C15']['data'] = c15

    RGB = AirMass(c08,c10,c12,c13)
    plts['AirMass']['data'] = RGB

    RGB = DayCloudPhase(c13,c02,c05)
    plts['DayCloudPhase']['data'] = RGB

    RGB = NighttimeMicrophysics(c07,c13,c15)
    plts['NightMicrophysics']['data'] = RGB

    RGBwv = DifferentialWaterVapor(c08,c10)
    plts['DifferentialWV']['data'] = RGBwv

    return

def create_timestrings(ds):
    tstamp = datetime.strptime(ds.attrs['time_coverage_start'][:-3],'%Y-%m-%dT%H:%M:%S')
    tst = datetime.strftime(tstamp, '%b %d, %Y -- %H:%M UTC')
    pfname = datetime.strftime(tstamp, 'satellite-mosaic_%Y-%m-%d-%H%MZ.png')
    return tst, pfname

In [ ]:
# @title Create satellite file, download files, then make plots
for d in ('sat_data','images'):
  os.makedirs(os.path.join('/content',d), exist_ok=True)

from goes2go import GOES
GCONUS = GOES(satellite=16, product="ABI-L1b-RadC", domain='C')
GCONUSALL = GOES(satellite=16, product="ABI-L2-MCMIPC", domain='C')
#GMESO = GOES(satellite=16, product="ABI-L1b-RadC", domain='M')
df = GCONUS.df(start=data_start, end=data_end)
dfall = GCONUS.df(start=data_start, end=data_end)
print(f'downloading satellite data from {data_start} to {data_end}')
SAT = GCONUS.timerange(start=data_start, end=data_end, save_dir = '/content/sat_data' )
SATALL = GCONUSALL.timerange(start=data_start, end=data_end, save_dir = '/content/sat_data' )

VIS = SAT.query('band == 2')
VIS = VIS.reset_index(drop = True)
ULWV = SAT.query('band == 8')
ULWV = ULWV.reset_index(drop = True)
MLWV = SAT.query('band == 9')
MLWV = MLWV.reset_index(drop = True)
LLWV = SAT.query('band == 10')
LLWV = LLWV.reset_index(drop = True)
IR = SAT.query('band == 13')
IR = IR.reset_index(drop = True)
IR15 = SAT.query('band == 15')
IR15 = IR15.reset_index(drop = True)
# The plts dictionary contains plot settings for each channel
# A 'data' key is created to point to the data array that's to be plotted
# The data values get updated with each file iteration

plts = {}
plts['C01'] = {'cmap':'Greys_r','vmn':0,'vmx':1,'title':'Channel 01 Visible'}
plts['C02'] = {'cmap':'Greys_r','vmn':0,'vmx':1,'title':'Channel 02 Visible'}
plts['C02i'] = {'cmap':'Greys_r','vmn':0,'vmx':18,'title':'Channel 02 Visible', 'latv': None, 'lonv': None}
plts['C05'] = {'cmap':'cod_ir','vmn':-110.0,'vmx':56.0,'title':'Channel 5 IR'}
plts['C07'] = {'cmap':'cod_ir','vmn':0,'vmx':1,'title':'Channel 7 IR'}
plts['C08'] = {'cmap':'wv_cmap','vmn':-109.0,'vmx':0.0,'title':'Channel 8 W/V'}
plts['C09'] = {'cmap':'wv_cmap','vmn':-109.0,'vmx':0.0,'title':'Channel 9 W/V'}
plts['C10'] = {'cmap':'wv_cmap','vmn':-109.0,'vmx':0.0,'title':'Channel 10 W/V'}
plts['C12'] = {'cmap':'cod_ir','vmn':-110.0,'vmx':56.0,'title':'Channel 12 IR'}
plts['C13'] = {'cmap':'cod_ir','vmn':-110.0,'vmx':56.0,'title':'Channel 13 IR'}
plts['C15'] = {'cmap':'cod_ir','vmn':-110.0,'vmx':56.0,'title':'Channel 15 IR'}
plts['ALL'] = {'cmap':'cod_ir','vmn':-110.0,'vmx':56.0,'title':'All'}
plts['C02i']['filelist'] = list(VIS['file'])
plts['C08']['filelist'] = list(ULWV['file'])
plts['C09']['filelist'] = list(MLWV['file'])
plts['C10']['filelist'] = list(LLWV['file'])
plts['C13']['filelist'] = list(IR['file'])
plts['C15']['filelist'] = list(IR15['file'])
plts['ALL']['filelist'] = list(SATALL.file)
plts['AirMass'] = {'cmap':'Greys_r','vmn':0,'vmx':1,'title':'AirMass'}
plts['DayCloudPhase'] = {'cmap':'Greys_r','vmn':0,'vmx':1,'title':'Day Cloud Phase'}
plts['NightMicrophysics'] = {'cmap':'Greys_r','vmn':0,'vmx':1,'title':'Nighttime Microphysics'}
plts['DifferentialWV'] = {'cmap':'Greys_r','vmn':0,'vmx':1,'title':'Differential W/V'}
plts['SplitWindowDifference'] = {'cmap':'Greys_r','vmn':0,'vmx':1,'title':'SplitWindowDifference'}

print("Now creating plots...")

for i,f in enumerate(plts['ALL']['filelist']):
    if 'C02i' in products:
      fv = plts['C02i']['filelist'][i]
      dsv = xr.open_dataset(os.path.join('/content/sat_data',fv))
      dsv = calc_latlon(dsv)
      ((x1v,x2v), (y1v, y2v)) = get_xy_from_latlon(dsv,lats,lons)
      subsetv = dsv.sel(x=slice(x1v, x2v), y=slice(y2v, y1v))
      latv = subsetv.lat.data
      lonv = subsetv.lon.data
      datv = subsetv['Rad'].data
      gamma = 2.2
      datv = np.power(datv,1/gamma)
      plts['C02i']['data'] = datv

    ds = xr.open_dataset(os.path.join('/content/sat_data',f))
    tstring, plot_filename = create_timestrings(ds)
    ds = calc_latlon(ds)
    ((x1,x2), (y1, y2)) = get_xy_from_latlon(ds,lats,lons)
    subset = ds.sel(x=slice(x1, x2), y=slice(y2, y1))
    lat = subset.lat.data
    lon = subset.lon.data

    extract_channels(subset);

#------------------------------------------------------------
#    Plotting routine below
#------------------------------------------------------------

    fig, axes = plt.subplots(2,3,figsize=(16,9),subplot_kw={'projection': ccrs.PlateCarree()})

    plt.rcParams['font.size'] = 14
    plt.rcParams['font.weight'] = 'bold'
    plt.rcParams['axes.titlesize'] = 18
    plt.rcParams['figure.titlesize'] = 24
    plt.rcParams['figure.titleweight'] = 'bold'
    plt.rcParams['axes.titleweight'] = 'bold'
    plt.rcParams['axes.titlecolor'] = '#000000'
    plt.rcParams['savefig.bbox'] = 'tight'

    fig.suptitle(tstring)

    for y,a in zip(products,axes.ravel()):
      plot = plts[y]
      a.set_extent(plot_extent, crs=ccrs.PlateCarree())
      a.set_aspect(1.25)
      if y == 'C02i':
        a.pcolormesh(lonv,latv, plot['data'], cmap=plot['cmap'],vmin=plot['vmn'],vmax=plot['vmx'])
      else:
        a.pcolormesh(lon,lat, plot['data'], cmap=plot['cmap'],vmin=plot['vmn'],vmax=plot['vmx'])
      a.axes.add_feature(cfeature.STATES, facecolor='none', edgecolor='white', linewidth=1.0)
      if plot_counties:
        a.axes.add_feature(COUNTY_SHAPEFILE, facecolor='none', edgecolor='#cccccc', linewidth=0.3)
      if plot_roads:
        a.axes.add_feature(ROAD_SHAPEFILE, facecolor='none', edgecolor='cyan', linewidth=0.5)
      a.set_title(plot['title'])

    plt.tight_layout()
    plt.savefig(os.path.join('/content/images',plot_filename))

## @markdown Separate Channels
#C02 = False # @param {type:"boolean"}
#C02i = False # @param {type:"boolean"}

#C08 = False # @param {type:"boolean"}
#C09 = False # @param {type:"boolean"}
#C10 = False # @param {type:"boolean"}
#C13 = False # @param {type:"boolean"}
#C14 = False # @param {type:"boolean"}
#C15 = False # @param {type:"boolean"}

## @markdown RGBs
#Airmass = False # @param {type:"boolean"}
#DayCloudPhase = False # @param {type:"boolean"}
#NightMicrophysics = False # @param {type:"boolean"}
#DifferentialWV = False # @param {type:"boolean"}